In [1]:
from scrap import Scraper
from RAG import RetrievalAugmentedGeneration
scrp = Scraper()

2024-07-30 12:11:50.956326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 12:11:50.978257: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 12:11:50.985040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 12:11:52.214392: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
url = "https://www.nu.edu.pk/Admissions/Scholarship"

In [3]:
scrp.scrape_website(url=url)

"Merit scholarship is awarded to the Top THREE position holders in each Group of each Examination Board. Scholarship is also offered to top three position holders in the merit list of NU admission merit list. This scholarship is for four years of un-interrupted studies with full course load and is equal to the full tuition fee.  Continuation of merit scholarship is subject to maintaining semester GPA of 3.0 or higher. The Higher Education Commission (HEC) provides liberal scholarships to students desirous of studying for the MS and PhD degrees at this university. The scholarship covers tuition fee, monthly stipend and book allowance. The Government of Punjab gives 42 scholarships to indigent students having domicile of Punjab. Newly admitted students in any campus of the university can apply for this scholarship. The scholarship is for 4-year undergraduate studies and covers some portion of the tuition fee. The remaining tuition fee can be given by the University as Qarz-e-Hasna. The S

In [7]:
rag = RetrievalAugmentedGeneration()
rag.Load_embed()

In [8]:
rag.Load_LLM()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [9]:
print(rag.generateResponse(prompt="Summer"))

1. Visit the University's website at admissions.nu.edu.pk or www.nu.edu.pk to access the online application.
        2. Follow the instructions provided to fill out your personal details, educational background, and select your desired program (BBA, BS, MS, MBA, or PhD).
        3. Submit the necessary documents, including your No Objection Certificate (for foreign nationals), SAT or GRE scores (if applicable), and any other required documents.
        4. Pay the application fee and submit your application.</s>


In [18]:
print(rag.window_mem.buffer_as_str)

Human: How are to apply for univeristy?
AI: 1. Visit the University's website at admissions.nu.edu.pk or www.nu.edu.pk to access the online application.
        2. Follow the instructions provided to fill out your personal details, educational background, and select your desired program (BBA, BS, MS, MBA, or PhD).
        3. Submit the necessary documents, including your No Objection Certificate (for foreign nationals), SAT or GRE scores (if applicable), and any other required documents.
        4. Pay the application fee and submit your application.</s>
